# Intro

## REQUERIMENTS
torch
torchvision
httpimport
scipy
sklearn
minepy

# Data Enginiering

In [ ]:
import pandas as pd
import numpy as np


## Load your dataset
data= pd.read_csv('your_dataset.csv') # For CSV
data= pd.read_excel('your_file.xlsx') # For Excel

## Make dataset more readable
# Rename specific columns
data = data.rename(columns={
    'OldColumn1': 'NewColumn1',
    'OldColumn2': 'NewColumn2'
})
# Verify changes
data.head()


## Ensure correct data interpretation
# Convert string to datetime
data['Date_Column'] = pd.to_datetime(data['Date_Column'])
# Convert string to numeric
data['Amount'] = pd.to_numeric(data['Amount'])
# Check data types
data.dtypes


## Deal with incomplete data
# Fill missing values with mean
data['Amount'] = data['Amount'].fillna(data['Amount'].mean())
# Or drop rows with any missing values
data= data.dropna()
# Verify changes
data.info()


## Keep data unique
# Remove duplicate rows
data = data.drop_duplicates()
# Remove duplicates based on specific columns
data = data.drop_duplicates(subset=['Columnl1', 'Column2'])
print(f'Remaining rows: {len(data)}')



## Ensure consitent formatting
# Convert text to lowercase
data['Category' ] = data['Category'].str.lower()
# Remove leading/ trailing spaces
data['Category'] = data['Category'].str.strip()
data['Category'].unique()



## Find extrreme values using IQR method
# Calculate IQR
QI = data['Amount'].quantile(0.25)
Q3 = data['Amount'].quantile(0.75)
IQR = Q3-QI
# Set bounds
lower_bound = QI-1.5*IQR
upper_bound = Q3+1.5*IQR
# Find outliers
outliers = data[(data['Amount'] < lower_bound) | 
                (data['Amount'] > upper_bound)]


## Select specific data based on conditions:
# Filter by single condition
filtered_data = data[data['Amount']>100]
# Filter by multiple conditions
filtered_data = data[
    (data['Amount'] > 100) &
    (data['Category'] == 'electronics')
]


## Column Management
# Drop unnecessary columns
data = data.drop(columns=['Unnecesary1', 'Unnecesary2'])
# Reorder columns
data = data[['Important1', 'Important2', 'Important3']]
# List remaining columns
data.columns


## Save your cleaned dataset

data.to_csv('cleaned_data.csv', index=False)    # Save as CSV
data.to_excel('cleaned_data.xlsx', index=False) # Save as Excel

print('Data cleaning completed!')

# Primera red

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
"""
Perceptrón para estimar precio de viviendas. Una capa oculta con 5 procesadores con activación tangente hiperbólica y error cuadrático.
Dependencias: torch, minepy, captum, httpimport
"""
import httpimport

analisisprevio=True
parteazar=0.4
parteajuste=0.7
partevalidacion=0.15
modelo='lineal'
ocultos=4
nolineal='ReLU'
funcionfinal='Sigmoid'
velocidad=0.1
analizaresiduos=False
verejemplos=False

###################################################################################################
with httpimport.remote_repo('https://personales.unican.es/crespoj/redes/redespytorch.zip'):
    import lectura
    import preproceso
    import particion
    import tipored
    import ajuste
    import registro
    import lecturaurl
    from torch import nn
    import activaciones

#Cargamos los datos, todos
datos=lecturaurl.leelistas('practica1/casas.trn.txt')
numentradas=len(datos[0])-1
varnoms=['criminalidad','residencial','industrial','rio','polucion','habitaciones','casas-viejas','distancia-trabajo',
         'autovias','impuestos','ratio-aula','negr@s','pobreza','precio']

#Descripción numÃ©rica
if analisisprevio:
    lectura.estadistica(varnoms,datos)

#Descripción gráfica
frel,vals=lectura.grafica(varnoms,datos,analisisprevio)

#Preproceso a distribución normal media 0 varianza 1. Hay otras posibilidades. Mira el fichero preproceso
datot=preproceso.mediavar(datos)

#Conjuntos de ajuste, validación y muestra. Parte al azar y parte por agrupamiento
tea,tsa,tev,tsv,tep,tsp=particion.azaryestrat(datot,parteazar,parteajuste,partevalidacion)

def modula(mod):
    if mod in dir(nn):
        return getattr(nn,mod)
    if mod in dir(activaciones):
        return getattr(activaciones,mod)
    return mod

nolineal=modula(nolineal)
funcionfinal=modula(funcionfinal)
argums={'lineal':(numentradas,1),'cuasilineal':(numentradas,ocultos,1),'perceptron':(numentradas,[ocultos],1,nolineal,funcionfinal)}
red=getattr(tipored,modelo)(*(argums[modelo]))
#).cuda(device)
print(red)
print('\nAjuste')
err,red,_=ajuste.ajustar(red,tea,tsa,tev,tsv,tep,tsp,kaj=velocidad)
print('Error medio cuadrático final en conjunto de prueba',err.item())
red.eval()
tepor=registro.residuos(tep,tsp,red,analizaresiduos)

if verejemplos:
#Vemos un caso concreto
    registro.sacarejemplos(tep,tsp,tepor,varnoms,frel,vals,red,False)
